# **Compensador dinâmico**

Já aprendemos a projetar um regulador, que é um conjunto de ganhos que calcula a ação de controle usando os estados do sistema.

Também aprendemos a projetar um observador de estados, que é um sistema dinâmico cuja função é fornecer uma estimativa dos estados reais do sistema a partir do sinal de saída.

O compensador dinâmico é a junção destas duas ideias no mesmo sistema.

A Figura abaixo esquematiza todos os subsistemas e rotas de sinal com a estratégia adotada.
<p>
<img src="Fig7.35.svg" width="80%">
</p>

No final, nosso controlador projetado com essas abordagens é um único sistema, cujas equações de estado são:
$$ 
\begin{align*}
    \dot{\mathbf{x}}_e &= \mathbf{(A-BK-LC)}\mathbf{x}_e+\mathbf{L}y\\
    u &= \mathbf{-K}\mathbf{x}_e
\end{align*}
$$
onde $\mathbf{x}_e$ é a estimativa dos estados do sistema pelo observador, $\mathbf{K}$ e $\mathbf{L}$ são respectivamente os ganhos do regulador e do observador. Note que a entrada do controlador é a saída da planta. Da mesma forma, o sinal de saída do controlador é o sinal de controle $u$, que vai para a entrada da planta. 

Perceba, que estas duas equações permitem realizar o controlador como um sistema entrada-saída normal, com uma função de transferência que basicamente dispensa as equações de estado, para fins de implementação. Esta função é:
$$ 
\begin{align*}
    C(s) &= -\mathbf{K}(s\mathbf{I-A+BK+LC})^{-1}\mathbf{L}
\end{align*}
$$


O sistema completo (planta+controlador) pode ser simulados usando um único conjunto de equações de estados:
$$
\begin{align*}
\dot{\mathbf{x}} &= \mathbf{F}\mathbf{x}-\mathbf{GK}\mathbf{x_e}\\
\dot{\mathbf{x}}_e &= \mathbf{(F-GK-LH)}\mathbf{x}_e+\mathbf{LHx}\\
y&= \mathbf{Hx}
\end{align*}
$$


É possível demonstrar que os pólos de malha fechada compensado por um controlador dessa natureza são exatamente os pólos projetados pelo regulador de estados completo (isto é, considerando realimentação dos estados verdadeiros, mesmo que na prática não vá ser assim no final) e o pólos alocados para o observador. 

Em outras palavras, o polinômio de malha fechada quando usamos um compensador que combina regulador+observador é simplesmente:
$$
\begin{align*}
    \alpha_{\text{mf}}(s) = \alpha_{\text{reg}}(s)\cdot \alpha_{\text{obs}}(s)
\end{align*}
$$
onde "mf", "reg" e "obs" indicam respectivamente "malha fechada", "regulador" e "observador".

O fato do projeto dos dois sistemas não "misturar" os pólos é um fato notável, que permite que os projetos sejam feitos de forma independente. Isso é chamado de principío da separação em teoria de controle. 

## Implementação quando o observador é reduzido

Quando o observador usado é de ordem reduzida, as equações de estado do controlador sofrem algumas mudanças, para acomodar a suposição que um dos estados é medido e o particionamento de matrizes. 

A função de transferência com o observador reduzido é:
$$
\begin{align*}
C(s) &= \mathbf{C}_r(s\mathbf{I}-\mathbf{A}_r)^{-1}\mathbf{B}_r+D_r
\end{align*}
$$
onde:
$$
\begin{align*}
\mathbf{A}_r &= \mathbf{F}_{bb}-\mathbf{LF}_{ab}-(\mathbf{G}_{b}-\mathbf{LG}_{a})\mathbf{K}_b\\
\mathbf{B}_r &= \mathbf{A}_{r}\mathbf{L}+\mathbf{F}_{ba}-\mathbf{L}\mathbf{F}_{aa}-(\mathbf{G}_{b}-\mathbf{LG}_{a}){K}_a\\
\mathbf{C}_r &= -\mathbf{K}_b\\
D_r &= -K_a-\mathbf{K}_b\mathbf{L}
\end{align*}
$$


No caso, as matrizes $\mathbf{F}_{aa}$, $\mathbf{F}_{ab}$, $\mathbf{F}_{ba}$, $\mathbf{G}_{a}$ e $\mathbf{G}_{b}$ são provenientes do particionamento das matrizes do sistema, conforme o projeto do observador reduzido. 

A matriz $\mathbf{K}_{b}$ e o ganho ${K}_{a}$ são provenientes do particionamento do vetor de ganhos do regulador. O ganho $K_a$ é o ganho associado ao estado diretamente medido (i.e. a saída) e $\mathbf{K}_{b}$ é o restante do vetor projetado. 

<p align="center">
<img src="Ex7.28.svg" width="80%">
</p>

Vamos resolver usando as funções diretas de alocação de polos

In [1]:
import numpy as np
import control as ct

In [2]:
# Define as matrizes
F = np.array([[0,1],[0,0]],dtype=float)
G = np.array([[0],[1]],dtype = float)
H = np.array([[1,0]],dtype=float)
H

array([[1., 0.]])

In [3]:
# Aloca os polos do regulador
pr = -0.707+0.707j
K = ct.acker(F,G,[pr,np.conj(pr)])
K

array([[0.999698, 1.414   ]])

No livro os ganhos são $1$ e $\sqrt{2}$ exatamente. Você pode conferir com o Python que os resultados decimais estão bem próximos.

In [4]:
# Projeto do observador
wn = 5
xi = 0.5
po = -xi*wn+1j*wn*np.sqrt(1-xi**2)
L = ct.acker(F.T, H.T, [po,np.conj(po)]).T
L

array([[ 5.],
       [25.]])

In [5]:
C = ct.ss(F - G@K - L@H,           L,           -K,              0)
print(C)
ct.tf(C)

<LinearIOSystem>: sys[2]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (2): ['x[0]', 'x[1]']

A = [[ -5.         1.      ]
     [-25.999698  -1.414   ]]

B = [[ 5.]
     [25.]]

C = [[-0.999698 -1.414   ]]

D = [[0.]]



TransferFunction(array([-40.34849, -24.99245]), array([ 1.      ,  6.414   , 33.069698]))

<p align="center">
<img src="Ex7.30.svg" width="80%">
</p>

In [6]:
# Expandir o numerador para achar os coeficientes
den = np.convolve([1,0],[1,2])
den = np.convolve(den,[1,8])
print(den)

[ 1 10 16  0]


In [7]:
# Assim, as matrizes serão:
F = np.array([[-10,1,0],[-16,0,1],[0,0,0]])
G = np.array([[0],[0],[10]])
H = np.array([[1,0,0]])
J = 0

sys = ct.ss(F,G,H,J)
print(sys)


<LinearIOSystem>: sys[4]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (3): ['x[0]', 'x[1]', 'x[2]']

A = [[-10.   1.   0.]
     [-16.   0.   1.]
     [  0.   0.   0.]]

B = [[ 0.]
     [ 0.]
     [10.]]

C = [[1. 0. 0.]]

D = [[0.]]



In [8]:
# Posicionamento de polos
pc = [-1.42, -1.04+2.14j,-1.04-2.14j]
pe = [-4.25, -3.13+6.41j,-3.13-6.41j]

K = ct.acker(F,G,pc)
L = ct.acker(F.T,H.T,pe).T

print(K)
print(L)

[[-46.4109096   5.76148    -0.65     ]]
[[  0.51   ]
 [ 61.49   ]
 [216.26125]]


In [9]:
# Montagem do controlador e função de transferência
Fc = F-G@K-L@H
Gc = L
Hc = -K

comp = ct.ss(Fc,Gc,Hc,0)
print(comp)

C = ct.tf(comp)
C

<LinearIOSystem>: sys[5]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (3): ['x[0]', 'x[1]', 'x[2]']

A = [[-10.51       1.         0.      ]
     [-77.49       0.         1.      ]
     [247.847846 -57.6148     6.5     ]]

B = [[  0.51   ]
     [ 61.49   ]
     [216.26125]]

C = [[46.4109096 -5.76148    0.65     ]]

D = [[0.]]



TransferFunction(array([-190.0340288 , -482.20028555, -173.85034277]), array([   1.      ,    4.01    ,   66.7898  , -146.001298]))

In [15]:
tab=ct.damp(C)
ct.zeros(C)

_____Eigenvalue______ Damping___ Frequency_
    -2.943    +8.317j     0.3336      8.822
    -2.943    -8.317j     0.3336      8.822
     1.876                     1     -1.876


array([-2.10227638+0.j, -0.43516541+0.j])

In [18]:
# Analisa o sistema em malha fechada
#F1 = np.concatenate((F,-G@K),axis=1)
#F2 = np.concatenate((L@H, F-G@K-L@H),axis=1)
Fmf = np.block([[F,-G@K],[L@H,F-G@K-L@H]])
#Fmf = np.concatenate((F1,F2),axis=0)
Gmf = np.zeros((2*G.size,1))
Hmf = np.concatenate((H,np.zeros((1,3))),axis=1)
Hmf = np.block([H,np.zeros((H.shape))])
MF = ct.ss(Fmf,Gmf,Hmf,0)
print(MF)

<LinearIOSystem>: sys[9]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (6): ['x[0]', 'x[1]', 'x[2]', 'x[3]', 'x[4]', 'x[5]']

A = [[-10.         1.         0.         0.         0.         0.      ]
     [-16.         0.         1.         0.         0.         0.      ]
     [  0.         0.         0.       464.109096 -57.6148     6.5     ]
     [  0.51       0.         0.       -10.51       1.         0.      ]
     [ 61.49       0.         0.       -77.49       0.         1.      ]
     [216.26125    0.         0.       247.847846 -57.6148     6.5     ]]

B = [[0.]
     [0.]
     [0.]
     [0.]
     [0.]
     [0.]]

C = [[1. 0. 0. 0. 0. 0.]]

D = [[0.]]



In [19]:
tab=ct.damp(MF)

_____Eigenvalue______ Damping___ Frequency_
     -3.13     +6.41j     0.4388      7.133
     -3.13     -6.41j     0.4388      7.133
     -4.25                     1       4.25
     -1.04     +2.14j     0.4371      2.379
     -1.04     -2.14j     0.4371      2.379
     -1.42                     1       1.42


In [12]:
ct.zeros(MF)

array([], dtype=float64)